Working with the med3pa Subpackage
-----------------------------------------

This tutorial guides you through the process of setting up and running comprehensive experiments using the `med3pa` subpackage. It includes steps to execute MED3pa experiment with `Med3paExperiment` and the combination of MED3pa and Detectron using `Med3paDetectronExperiment`.


## Running the MED3pa Experiment


### Step 1: Setting up the Datasets
First, configure the `DatasetsManager`. In the case of MED3pa only experiment you only need to set the DatasetManager with either `testing` and `reference` dataset:


In [1]:
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from MED3pa.datasets import DatasetsManager

# Initialize the DatasetsManager
datasets = DatasetsManager()

# Load datasets for reference, and testing
datasets.set_from_file(dataset_type="reference", file='./data/test_data.csv', target_column_name='Outcome')
datasets.set_from_file(dataset_type="testing", file='./data/test_data_shifted_0.6.csv', target_column_name='Outcome')


## Step 2: Configuring the Model
Next, utilize the `ModelFactory` to load a pre-trained model, and set it as the base model for the experiment. Alternatively, you can train your own model and use it:


In [ ]:
from MED3pa.models import BaseModelManager, ModelFactory

# Initialize the model factory and load the pre-trained model
factory = ModelFactory()
model = factory.create_model_from_pickled("./models/diabetes_xgb_model.pkl")

# Set the base model using BaseModelManager
base_model_manager = BaseModelManager()
base_model_manager.set_base_model(model=model)


## Step 3: Running the MED3pa Experiment
Execute the MED3PA experiment with the specified datasets and base model. You can also specify other parameters as needed. See the documentation of the subpackage for more information about the parameters.

The experiment outputs two structure one for the reference set and the other for the testing set, both containing files indicating the extracted profiles at different declaration rates, the performance of the model on these profiles..etc.


In [ ]:
from MED3pa.med3pa import Med3paExperiment
from MED3pa.med3pa.uncertainty import AbsoluteError

# Define parameters for the experiment
ipc_params = {'n_estimators': 100}
apc_params = {'max_depth': 3}
med3pa_metrics = ['Auc', 'Accuracy', 'BalancedAccuracy']

# Execute the MED3PA experiment
reference_results, test_results = Med3paExperiment.run(
                                    datasets_manager=datasets,
                                    base_model_manager=base_model_manager,
                                    uncertainty_metric=AbsoluteError,
                                    ipc_type='RandomForestRegressor',
                                    ipc_params=ipc_params,
                                    apc_params=apc_params,
                                    samples_ratio_min=0,
                                    samples_ratio_max=50,
                                    samples_ratio_step=5,
                                    med3pa_metrics=med3pa_metrics,
                                    evaluate_models=True,
                                    models_metrics=['MSE', 'RMSE']
                                )


Running MED3pa Experiment on the reference set:


AttributeError: 'IPCModel' object has no attribute 'regressors_mapping'

## Step 4: Analyzing and Saving the Results
After running the experiment, you can analyze and save the results using the returned `Med3paResults` instance:


In [ ]:
# Save the results to a specified directory
reference_results.save(file_path='./med3pa_experiment_results/reference')
test_results.save(file_path='./med3pa_experiment_results/test')


## Running the MED3pa and Detectron Experiment
You can also run an experiment that combines the forces of Detectron in covariate shift detection with MED3pa problematic profiles extraction using the `Med3paDetectronExperiment` class. To be able to run this experiment, all datasets of the `DatasetsManager` should be set, alongside the ``BaseModelManager``. This experiment will run MED3pa experiment on the `testing` and `reference` sets and then run the `detectron` experiment on the `testing` set as a whole, and then on the **extracted profiles** from MED3pa:


In [ ]:
# We will add the training and validation set to the datasets
datasets.set_from_file(dataset_type="training", file='./data/train_data.csv', target_column_name='Outcome')
datasets.set_from_file(dataset_type="validation", file='./data/val_data.csv', target_column_name='Outcome')


In [ ]:
from MED3pa.med3pa import Med3paDetectronExperiment
from MED3pa.detectron.strategies import EnhancedDisagreementStrategy

# Execute the integrated MED3PA and Detectron experiment
reference_det_results, test_det_results, detectron_results = Med3paDetectronExperiment.run(
    datasets=datasets,
    base_model_manager=base_model_manager,
    uncertainty_metric=AbsoluteError,
    samples_size=20,
    ensemble_size=10,
    num_calibration_runs=100,
    patience=3,
    test_strategies=EnhancedDisagreementStrategy,
    allow_margin=False,
    margin=0.05,
    ipc_params=ipc_params,
    apc_params=apc_params,
    samples_ratio_min=0,
    samples_ratio_max=50,
    samples_ratio_step=5,
    med3pa_metrics=med3pa_metrics,
    evaluate_models=True,
    models_metrics=['MSE', 'RMSE']
)


Running MED3pa Experiment on the reference set:
IPC Model training completed.
APC Model training completed.
Confidence scores calculated for minimum_samples_ratio =  0
Results extracted for minimum_samples_ratio =  0
Confidence scores calculated for minimum_samples_ratio =  5
Results extracted for minimum_samples_ratio =  5
Confidence scores calculated for minimum_samples_ratio =  10
Results extracted for minimum_samples_ratio =  10
Running MED3pa Experiment on the reference set:
IPC Model training completed.
APC Model training completed.
Confidence scores calculated for minimum_samples_ratio =  0
Results extracted for minimum_samples_ratio =  0
Confidence scores calculated for minimum_samples_ratio =  5
Results extracted for minimum_samples_ratio =  5
Confidence scores calculated for minimum_samples_ratio =  10
Results extracted for minimum_samples_ratio =  10
Running Global Detectron Experiment:


running seeds: 100%|██████████| 100/100 [00:10<00:00,  9.52it/s]


Detectron execution on reference set completed.


running seeds: 100%|██████████| 100/100 [00:11<00:00,  8.76it/s]


Detectron execution on testing set completed.
Running Profiled Detectron Experiment:
Running Detectron on Profile: *, Age <= -0.8201243877410889 & BloodPressure <= 0.4599476158618927 & Glucose <= -0.06547205429524183


...

Detectron execution on testing set completed.
Running Detectron on Profile: *, Age > -0.8201243877410889
Calibration record on reference set provided, skipping Detectron execution on reference set.


running seeds: 100%|██████████| 100/100 [00:10<00:00,  9.66it/s]


Detectron execution on testing set completed.
Running Detectron on Profile: *, 
Calibration record on reference set provided, skipping Detectron execution on reference set.


running seeds: 100%|██████████| 100/100 [00:10<00:00,  9.81it/s]


Detectron execution on testing set completed.


In [ ]:
# Save the results to a specified directory
reference_det_results.save(file_path='./med3pa_detectron_experiment_results/reference')
test_det_results.save(file_path='./med3pa_detectron_experiment_results/test')
detectron_results.save(file_path='./med3pa_detectron_experiment_results/detectron')
